<a href="https://colab.research.google.com/github/dj20javad/VideoColorizerColab.ipynb/blob/main/VideoColorizerColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **<font color='blue'> Video Colorizer </font>**

#◢ DeOldify - Colorize your own videos!


_FYI: This notebook is intended as a tool to colorize gifs and short videos, if you are trying to convert longer video you may hit the limit on processing space. Running the Jupyter notebook on your own machine is recommended (and faster) for larger video sizes._

####**Credits:**

Big special thanks to:

Robert Bell for all his work on the video Colab notebook, and paving the way to video in DeOldify!

Dana Kelley for doing things, breaking stuff & having an opinion on everything.



---


#◢ Verify Correct Runtime Settings

**<font color='#FF000'> IMPORTANT </font>**

In the "Runtime" menu for the notebook window, select "Change runtime type." Ensure that the following are selected:
* Runtime Type = Python 3
* Hardware Accelerator = GPU


#◢ Git clone and install DeOldify

In [1]:
# 1) کلون کردن DeOldify (فقط یک بار)
!git clone https://github.com/jantic/DeOldify.git DeOldify

Cloning into 'DeOldify'...
remote: Enumerating objects: 2620, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 2620 (delta 64), reused 46 (delta 46), pack-reused 2514 (from 3)
Receiving objects: 100% (2620/2620), 69.73 MiB | 25.93 MiB/s, done.
Resolving deltas: 100% (1174/1174), done.


In [2]:
# 2) ورود به پوشه DeOldify
%cd DeOldify

/content/DeOldify


In [ ]:
# 3) نصب وابستگی‌ها
!pip install -r requirements-colab.txt

In [ ]:
!pip uninstall -y fastai
!pip install fastai==1.0.61

In [13]:
!mkdir -p /content/DeOldify/models

In [ ]:
!wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O /content/DeOldify/models/ColorizeVideo_gen.pth

In [5]:
import torch, functools
import fastai
import torch
import torch.serialization

torch.serialization.add_safe_globals([
    torch.nn.modules.container.Sequential,
        torch.nn.modules.conv.Conv2d,
])

In [6]:

# اضافه کردن کلاس‌ها/توابع قدیمی موردنیاز DeOldify
from fastai.basic_train import Recorder
from torch.nn.utils import spectral_norm

torch.serialization.add_safe_globals([
    functools.partial,
    Recorder,
    spectral_norm
])

In [9]:
from deoldify.visualize import get_video_colorizer
from deoldify import device
from deoldify.device_id import DeviceId

In [8]:
import torch

# ذخیره نسخه اصلی
_orig_load = torch.load

# بازنویسی torch.load
def patched_load(*args, **kwargs):
    kwargs['weights_only'] = False
    return _orig_load(*args, **kwargs)

torch.load = patched_load

In [10]:
if torch.cuda.is_available():
    print("✅ Using GPU")
    device.set(DeviceId.GPU0)
else:
    print("✅ Using CPU (no GPU found)")
    device.set(DeviceId.CPU)

✅ Using CPU (no GPU found)


In [15]:
colorizer = get_video_colorizer()

In [17]:
!mkdir -p /content/DeOldify/video/source
!cp 0bfd3267-10f1-42b3-85ec-39d9dc2028ba.mp4 /content/DeOldify/video/source/

cp: cannot stat '0bfd3267-10f1-42b3-85ec-39d9dc2028ba.mp4': No such file or directory


In [18]:
colorizer.colorize_from_file_name(
    file_name='0bfd3267-10f1-42b3-85ec-39d9dc2028ba.mp4',   # ویدیو ورودی
    render_factor=21,        # کیفیت رندر
    watermarked=False         # واترمارک
)

Exception: Video at path specfied, video/source/0bfd3267-10f1-42b3-85ec-39d9dc2028ba.mp4 could not be found.

#◢ Instructions

### source_url
Type in a url hosting a video from YouTube, Imgur, Twitter, Reddit, Vimeo, etc.  Many sources work!  GIFs also work.  Full list here: https://ytdl-org.github.io/youtube-dl/supportedsites.html NOTE: If you want to use your own video, upload it first to a site like YouTube.

### render_factor
The default value of 21 has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-). This determines resolution at which the color portion of the video is rendered. Lower resolution will render faster, and colors also tend to look more vibrant. Older and lower quality film in particular will generally benefit by lowering the render factor. Higher render factors are often better for higher quality videos and inconsistencies (flashy render) will generally be reduced, but the colors may get slightly washed out.

### watermarked
Selected by default, this places a watermark icon of a palette at the bottom left corner of the image.  This is intended to be a standard way to convey to others viewing the image that it is colorized by AI. We want to help promote this as a standard, especially as the technology continues to improve and the distinction between real and fake becomes harder to discern. This palette watermark practice was initiated and lead by the company MyHeritage in the MyHeritage In Color feature (which uses a newer version of DeOldify than what you're using here).

### How to Download a Copy
Simply right click on the displayed video and click "Save video as..."!

## Pro Tips
1. If a video takes a long time to render and you're wondering how well the frames will actually be colorized, you can preview how well the frames will be rendered at each render_factor by using the code at the bottom. Just stop the video rendering by hitting the stop button on the cell, then run that bottom cell under "See how well render_factor values perform on a frame here". It's not perfect and you may still need to experiment a bit especially when it comes to figuring out how to reduce frame inconsistency.  But it'll go a long way in narrowing down what actually works.
2. If videos are taking way too much time for your liking, running the Jupyter notebook VideoColorizer.ipynb on your own machine (with DeOldify installed) will generally be much faster (as long as you have the hardware for it).   
3. Longer videos (running multiple minutes) are going to have a rough time on Colabs. You'll be much better off using a local install of DeOldify instead in this case.

## Troubleshooting
The video player may wind up not showing up, in which case- make sure to wait for the Jupyter cell to complete processing first (the play button will stop spinning).  Then follow these alternative download instructions

1. In the menu to the left, click Files
2. If you don't see the 'DeOldify' folder, click "Refresh"
3. By default, rendered video will be in /DeOldify/video/result/

If a video you downloaded doesn't play, it's probably because the cell didn't complete processing and the video is in a half-finished state.

#◢ Colorize!!

In [19]:
source_url = 'https://youtube.com/shorts/oCoQmGavdvE' #@param {type:"string"}
render_factor = 21  #@param {type: "slider", min: 5, max: 40}
watermarked = True #@param {type:"boolean"}

if source_url is not None and source_url !='':
    video_path = colorizer.colorize_from_url(source_url, '0bfd3267-10f1-42b3-85ec-39d9dc2028ba.mp4', render_factor, watermarked=watermarked)
    show_video_in_notebook(video_path)
else:
    print('Provide a video url and try again.')

[youtube] Extracting URL: https://youtube.com/shorts/oCoQmGavdvE
[youtube] oCoQmGavdvE: Downloading webpage
[youtube] oCoQmGavdvE: Downloading tv client config
[youtube] oCoQmGavdvE: Downloading player 093288cd-main
[youtube] oCoQmGavdvE: Downloading tv player API JSON
[youtube] oCoQmGavdvE: Downloading ios player API JSON
[youtube] oCoQmGavdvE: Downloading m3u8 information
[info] Testing format 609
[info] oCoQmGavdvE: Downloading 1 format(s): 609+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 1
[download] Destination: video/source/0bfd3267-10f1-42b3-85ec-39d9dc2028ba.f609.mp4
[download] 100% of  527.21KiB in 00:00:00 at 1.45MiB/s                 
[download] Destination: video/source/0bfd3267-10f1-42b3-85ec-39d9dc2028ba.f140.m4a
[download] 100% of   86.66KiB in 00:00:00 at 322.47KiB/s 
[Merger] Merging formats into "video/source/0bfd3267-10f1-42b3-85ec-39d9dc2028ba.mp4"
Deleting original file video/source/0bfd3267-10f1-42b3-85ec-39d9dc2028ba.f609.mp4 (pass -k to

INFO:root:Video created here: video/result/0bfd3267-10f1-42b3-85ec-39d9dc2028ba.mp4


Video created here: video/result/0bfd3267-10f1-42b3-85ec-39d9dc2028ba.mp4


NameError: name 'show_video_in_notebook' is not defined

In [20]:
from google.colab import files

# مسیر ویدیوی خروجی (معمولاً توی video/result ذخیره میشه)
output_path = "video/result/0bfd3267-10f1-42b3-85ec-39d9dc2028ba.mp4"

# دانلود به سیستم شخصی
files.download(output_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## See how well render_factor values perform on a frame here

In [ ]:
for i in range(10,40,2):
    colorizer.vis.plot_transformed_image('video/bwframes/video/00001.jpg', render_factor=i, display_render_factor=True, figsize=(8,8))

---
#⚙ Recommended video and gif sources
* [/r/Nickelodeons/](https://www.reddit.com/r/Nickelodeons/)
* [r/silentmoviegifs](https://www.reddit.com/r/silentmoviegifs/)
* https://twitter.com/silentmoviegifs